# TP Programmation d'un réseau de neurones

Ce TP vise à construire votre propre réseau de neurones. Vous mettrez en place les fonctions de base qui permettent d'effectuer un calcul feed-forward d'un réseau de neurones ainsi que la backpropagation, pour ensuite effectuer un apprentissage sur le problème dit du XOR (ou exclusif). Ce TP permet de maîtriser les notions de base et comprendre les calculs effectués au sein d'un réseau de neurones lors d'une prédiction et lors de l'apprentissage. En revanche, par la suite, vous verrez que l'utilisation de bibliothèques dédiées comme Tensorflow/Keras est à privilégier : l'implémentation des codes est bien plus rapide et les algorithmes d'apprentissage sont optimisés pour tourner plus rapidement, exploiter les GPUs si votre machine en est équipée...

Dans ce TP, des cellules seront laissées à trous, il faudra les compléter suivant les consignes. Elles seront identifiées par le mot **Exercice**. Certaines seront suivies de cellules vérifications qui vous permettront de vérifier si le résultat codé correspond bien au résultat attendu, elles seront précédées du mot **Vérification**.

**Exercice** : Importer numpy et matplotlib.pyplot et les nommer avec les mots-clés np et plt

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

**Vérification** : Exécuter la cellule suivante. Une droite doit apparaître.

In [ ]:
plt.plot(np.arange(5))

## Calcul forward pass

Dans cette première partie, nous allons créer les fonctions permettant d'effectuer le calcul forward pass d'un réseau de neurones.

### Fonctions d'activation

**Exercice** : Commençons par coder les fonctions d'activation que nous allons utiliser. Nous allons principalement nous concentrer sur les fonctions d'activation ReLU (pour les couches intermédiaires du réseau) et sigmoïde (pour la sortie, problème de classification).

En exercice, coder la fonction "activation" ci-dessous qui prend en argument un vecteur $Z$ une chaîne de caractère, soit "ReLU", soit "sigmoide" et qui renvoie le vecteur sur lequel est appliqué la fonction d'activation.

On rappelle :
- $ReLU(Z) = max(Z,0)$
- $sigmoide(Z) = \frac{1}{1 + e^{-Z}}$

Pour la sigmoïde, je vous conseille de privilégier l'utilisation de la tangente hyperbolique pour effectuer le calcul, cela peut éviter des problèmes numérique dans le calcul de l'exponentiel. La sigmoïde peut effectivement s'exprimer à l'aide dans la fonction tanh.

In [ ]:
def activation(Z,fonction):
    
    if fonction == "ReLU":
        
        resultat = #A COMPLETER
    
    if fonction == "sigmoide":
        
        resultat = #A COMPLETER
    
    return resultat

**Vérification** : Exécutez la cellule suivante

In [ ]:
#NE PAS MODIFIER

Z = np.array([2.,10.,-5.,0.,-100.])

print(activation(Z,"ReLU"))
print(activation(Z,"sigmoide"))

### Calcul d'une couche de neurones

**Exercice** : On considère ici une couche de neurones, représentée par un vecteur $A$ (de $n \times N$ éléments représentant les $n$ neurones de la couche et $N$ représente le nombre d'exemples, car on peut avoir plusieurs exemples en même temps !). La couche suivante contiendra $m$ neurones, les poids reliant les deux couches seront représentés par une matrice $W$ de taille $m \times n$, et les biais par un vecteur de taille $m$.

Pour l'instant, on n'applique pas de fonction d'activation. On effectue le calcul linéaire : $Z = W.A + b$.

Vous devrez aussi renvoyer une variable "cache" qui contiendra simplement le triplet $(A,W,b,Z)$ qui vous avez en entrée. Cette variable peut paraître inutile à ce stade, mais elle sera en fait très utile dans le cadre de la backpropagation : souvenez-vous, il faut utiliser l'état de l'ensemble des couches du réseau pour effectuer le calcul de backpropagation.

**Hint** : Pour le calcul linéaire, la fonction np.dot devrait être utile.

In [ ]:
def one_layer_linear(A,W,b):
    
    Z = #A COMPLETER
    cache = #A COMPLETER
    
    return Z, cache

**Vérification** : Exécutez la cellule suivante :

In [ ]:
#NE PAS MODIFIER

np.random.seed(seed = 1)

A = np.random.rand(3,2)
W = np.random.rand(4,3)
b = np.random.rand(4,1)

Z, cache = one_layer_linear(A,W,b)

print(Z)
print(cache == (A,W,b,Z))

**Exercice** : Cette fois, on fait le calcul en entier, avec en plus la fonction d'activation. Notons $g$ la fonction d'activation, $A_{prev}$ le vecteur représentant la couche précédente, $A_{new}$ le résultat de la nouvelle couche, on a donc : $A_{new} = g(W.A_{prev} + b)$. La variable "fonction" contient une chaîne de caractère qui définit la fonction (soit "ReLU", soit "sigmoide" dans notre cas).

La variable cache devra contenir les mêmes quantités que précédemment.

**Hint** : Réutilisez les fonctions "activation" et "one_layer_linear" que vous avez codées précédemment.

In [ ]:
def one_layer_activation(A_prev,W,b,fonction):
    
    Z, cache = #A COMPLETER
    A_new = #A COMPLETER
        
    return A_new, cache

**Vérification** : Exécutez la cellule ci-dessous :

In [ ]:
#NE PAS MODIFIER

np.random.seed(seed = 1)

A_prev = np.random.rand(3,2)
W = np.random.rand(4,3)
b = np.random.rand(4,1)

Z, cache_old = one_layer_linear(A_prev,W,b)

A_new, cache_new = one_layer_activation(A_prev,W,b,"sigmoide")
print(A_new)

### Calcul du réseau de neurones entier

**Exercice** : Supposons que nous avons une liste de paramètres "List_W" et "List_b" qui contiennent l'ensemble des paramètres du réseau (poids et biais) pour chaque couche, ainsi qu'une entrée $X$ de taille $n_X \times N$, où $n_X$ est la dimension des inputs du réseau de neurones et $N$ est le nombre d'exemples.

"List_W" et "List_b" ont la même taille (le nombre de couches du réseau). On se donne aussi "List_activ" qui contient les fonctions d'activation à appliquer.

Codez la fonction ci-dessous qui applique le calcul feed_forward en entier, sur l'ensemble des couches.

La variable "caches" sera une liste qui contiendra l'ensemble des caches calculés à chaque couche du réseau. La variable Y_pred contiendra la sortie du réseau.

**Hints** :
- Cette fonction sera composée d'une boucle principale qui parcourera l'ensemble des couches du réseau.
- Réutilisez bien sûr la fonction "one_layer_activation" pour chaque couche.
- Pour connaître le nombre de couches, regardez la taille de l'une des listes de paramètres en entrée (np.size sera utile).
- Ajouter des éléments elem dans une liste l se fait en utilisant l.append(elem).

In [ ]:
def feed_forward(X,list_W,list_b,list_activations):
    
    A = #A COMPLETER
    N_couches = #A COMPLETER
    caches = []
    
    for i in range(N_couches):
    
        #A COMPLETER
    
    Y_pred = A
    
    return Y_pred,caches

**Vérification** : Exécutez la cellule ci-dessous

In [ ]:
#NE PAS MODIFIER

np.random.seed(seed = 2)

X = np.random.rand(5,3)
list_W = [np.random.rand(4,5)-1/2,np.random.rand(4,4)-1/2,np.random.rand(2,4)-1/2]
list_b = [np.random.rand(4,1)-1/2,np.random.rand(4,1)-1/2,np.random.rand(2,1)-1/2]
list_activations = ["ReLU","ReLU","sigmoide"]

Y_pred,caches = feed_forward(X,list_W,list_b,list_activations)

print(Y_pred)
print(len(caches) == 3)

## Loss function

Nous allons implémenter maintenant la loss function que nous allons utiliser. Ici, ce sera la binary_cross_entropy. Nous allons faire de la classification multi-classes non exclusives avec $K$ classes.

Nous redonnons la formule ci-dessous, pour un vecteur de prédictions $\hat{Y}$ composé de $N$ exemples $\hat{y_ik}$ qui doit être comparé à un vecteur de vraies valeurs $Y$ composé d'exemples $y_{ik}$ (l'indice $i$ représente l'exemple $i$ et l'indice $k$ représente la classe $k$):

\begin{equation}
L(Y,\hat{Y}) = - \frac{1}{N} \sum_{i=1}^{N} \sum_{k=1}^{K} y_{ik} \log(\hat{y_{ik}}) + (1-y_{ik})\log(1-\hat{y_{ik}})
\end{equation}

**Exercice** : Codez la loss-function ci-dessous, où la variable Y_pred représente les prédictions.

**Hints** :
- Déterminez le nombre d'exemple à partir de la taille de Y (seconde dimension), la fonction shape sera utile
- Les fonctions np.sum et np.log seront utiles

In [ ]:
def loss_function(Y,Y_pred):
    
    N = #A COMPLETER
    
    loss = #A COMPLETER
    
    return loss
    

**Vérification** : Exécutez la cellule ci-dessous

In [ ]:
#NE PAS MODIFIER

np.random.seed(seed = 3)

Y = np.array([[1,0,1],[0,1,1]])
Y_pred = np.random.rand(2,3)

print(loss_function(Y,Y_pred))

## Backward propagation

Dans cette partie, nous allons programmer la backpropagation de l'erreur, qui permettra de calculer les gradients et ainsi mettre à jour les paramètres du réseau de neurones.

### Backward linéraire

Considérons une couche $l$. On rappelle le calcul feed_forward : $Z^{[l]} = W^{[l]}.A^{[l-1]} + b^{[l]}$.

Supposons déjà connue la dérivée de la fonction de coût par rapport à $Z$ (de manière récursive) $\frac{\partial L}{\partial Z^{[l]}}$.

Nous allons calculer $\frac{\partial L}{\partial W^{[l]}}$, $\frac{\partial L}{\partial b^{[l]}}$ et $\frac{\partial L}{\partial A^{[l-1]}}$. Les deux premières permettent d'obtenir le gradient des paramètres de la couche $l$. La dernière dérivée sera utilisée pour la récursivité de la backpropagation.

On rappelle les formules (correspondant aux moyennes sur l'ensemble des exemples, d'où la présence des transposées et de la somme pour b) :

\begin{equation}
\frac{\partial L}{\partial W^{[l]}} = \frac{\partial L}{\partial Z^{[l]}} \frac{\partial Z^{[l]}}{\partial W^{[l]}} = \frac{\partial L}{\partial Z^{[l]}} A^{[l-1]T}
\end{equation}

\begin{equation}
\frac{\partial L}{\partial b^{[l]}} = \sum_{i = 1}^{N} (\frac{\partial L}{\partial Z^{[l]}})_i 
\end{equation}

\begin{equation}
\frac{\partial L}{\partial A^{[l-1]}} = \frac{\partial L}{\partial Z^{[l]}} \frac{\partial Z^{[l]}}{\partial A^{[l-1]}} = W^{[l]T}\frac{\partial L}{\partial Z^{[l]}} 
\end{equation}




**Exercice** : Codez ci-dessous ces trois formules dans la fonction "linear_backward". En entrée, on prend dZ, correspondant à la dérivée de Z supposée connue, et le cache qui contient (A_prev,W,b,Z), d'où l'utilité de ce cache que nous avions défini dans les fonctions feed_forward.

**Hint** : Pour la somme, bien faire attention à la dimension sur laquelle on l'applique (dimension correspondant aux exemples), à utiliser avec le mot-clé "axis = ...". Utilisez bien aussi le mot clé "keepdims = True" pour pouvoir maintenir les opérations valides au niveau de la dimension.

In [ ]:
def linear_backward(dZ,cache):
    
    A_prev,W,b,Z = cache
    
    dW = #A COMPLETER
    db = #A COMPLETER
    dA_prev = #A COMPLETER
    
    return dA_prev,dW,db

**Vérification** : Exécutez la cellule ci-dessous

In [ ]:
#NE PAS MODIFIER

np.random.seed(seed = 3)

dZ = np.random.rand(3,2)

A_pred = np.random.rand(4,2)
W = np.random.rand(3,4)
b = np.random.rand(3,1)
Z = np.random.rand(3,2)

cache =  (A_pred,W,b,Z)

print(linear_backward(dZ,cache))

### Ajoutons l'activation

**Exercice** : Dans un premier temps, nous allons calculer les dérivées des fonctions d'activation que nous avons utilisées.

Nous rappelons que :
- Pour $g = ReLU$, $g'(x) = 0$ si $x < 0$ et $g'(x) = 1$ si $x \ge 0$.
- Pour $g = sigmoide$, $g'(x) = g(x)(1-g(x))$

**Hints** : 
- Ne mettez pas de condition dans ReLU de type if, then ! Utilisez directement l'écriture booléenne $Z > 0$.
- Pour la sigmoïde, réutilisez la fonction activation que vous aviez codée au tout début

In [ ]:
def derivate_activation(Z,activ):
    
    if activ == "ReLU":
        
        dg = #A COMPLETER
        
    if activ == "sigmoide":
        
        dg = #A COMPLETER
    
    return dg

**Vérification** : Exécutez la cellule suivante

In [ ]:
#NE PAS MODIFIER

np.random.seed(seed = 4)

Z = np.random.rand(4,2) - 1/2

print(derivate_activation(Z,"ReLU"))
print(derivate_activation(Z,"sigmoide"))

**Exercice** : On ajoute maintenant la dérivée de la fonction d'activation afin de calculer $\frac{\partial L}{\partial Z^{[l]}}$ à partir de $\frac{\partial L}{\partial A^{[l]}}$ (supposée connue par récursivité... vu qu'on la calcule avec la fonction précédente).

On rappelle la formule :

\begin{equation}
A^{[l]} = g(Z^{[l]})
\end{equation}

Ce qui donne donc :

\begin{equation}
\frac{\partial L}{\partial Z^{[l]}} = \frac{\partial L}{\partial A^{[l]}} \frac{\partial A^{[l]}}{\partial Z^{[l]}} = \frac{\partial L}{\partial A^{[l]}} g'(Z^{[l]})
\end{equation}

La fonction d'activation est donnée par activ.

**Hint** :
- La valeur de $Z^{[l]}$ sera utile, on rappelle qu'elle est justement dans le cache qui contient A_prev,W,b,Z
- Réutilisez les fonctions précédentes "linear_backward" pour retrouver les valeurs dA_prev, dW et db à partir du dZ que vous aurez calculé et derivate_activation pour avoir les dérivées des fonctions d'activation


In [ ]:
def activation_backward(dA,cache,activ):
    
    A_prev,W,b,Z = cache
    
    dZ = #A COMPLETER
    
    dA_prev,dW,db = #A COMPLETER
    
    return dA_prev,dW,db

**Vérification** : Exécutez la cellule ci-dessous

In [ ]:
np.random.seed(seed = 3)

dA = np.random.rand(3,2)

A_pred = np.random.rand(4,2)
W = np.random.rand(3,4)
b = np.random.rand(3,1)
Z = np.random.rand(3,2)

cache =  (A_pred,W,b,Z)

print(activation_backward(dA,cache,"sigmoide"))

### Et on fait remonter le long du réseau

On part maintenant de la fin du réseau et on fait remonter le long des couches.

**Exercice** : Appliquez la backpropagation ci-dessous. 

**Hints** :
- Il faut dans un premier initialiser en calculant la dérivée de la loss function pour la dernière couche :
\begin{equation}
\frac{\partial L}{\partial A^{[N_{couches}]}} = \frac{\partial L}{\partial \hat{Y}^{[l]}} = - \frac{1}{N} (\frac{Y}{\hat{Y}} - \frac{1 - Y}{1 - \hat{Y}}) 
\end{equation}

- On fait ensuite une boucle en remontant le sens des couches (notez l'utilisation de reversed(range(Ncouches))

- Utilisez les caches qui sont stockés sous forme de liste

- Stockez enfin les gradients des paramètres dans deux listes list_dW et list_db : attention de les insérez à chaque fois au début de la liste itérativement (et non à la fin) car on remonte les couches dans la backpropagation ! La fonction l.insert(0,var) sera utile pour cela, où l est la liste, 0 est la position (au début de la liste) et var est la variable à insérer.

In [ ]:
def backward_propagation(Y_pred,Y,caches,activations):
    
    list_dW = []
    list_db = []
    
    N_couches = #A COMPLETER
    N = #A COMPLTER (nombre d'exemples)
    Y = Y.reshape(np.shape(Y_pred))
    
    dA_cur = #A COMPLETER (initialisation : dérivée de la dernière couche)
    
    for l in reversed(range(N_couches)):

        dA_prev,dW,db = #A COMPLETER
        
        #A COMPLETER : insertion dans les listes list_dW, list_db
        
        dA_cur = #A COMPLETER
    
    return list_dW, list_db      

In [ ]:
#NE PAS MODIFIER

np.random.seed(seed = 2)

X = np.random.rand(5,3)
list_W = [np.random.rand(4,5)-1/2,np.random.rand(4,4)-1/2,np.random.rand(2,4)-1/2]
list_b = [np.random.rand(4,1)-1/2,np.random.rand(4,1)-1/2,np.random.rand(2,1)-1/2]
list_activations = ["ReLU","ReLU","sigmoide"]

Y_pred,caches = feed_forward(X,list_W,list_b,list_activations)

Y = np.array([[0,1,1],[1,0,1]])

print(backward_propagation(Y_pred,Y,caches,list_activations))

## Mise à jour des paramètres

**Exercice** : Une fois les gradients des paramètres calculés, il suffit de mettre à jour les anciens paramètres et les remplacer par les nouveaux. Codez la fonction qui prend la liste des anciens paramètres (list_w,list_b), la liste des gradients (list_dw,list_db) et le taux d'apprentissage alpha, pour renvoyer la liste des nouveaux paramètres suivant la formule de mise à jour du gradient :

\begin{equation}
W^{[l]} := W^{[l]} - \alpha \frac{\partial L}{\partial W^{[l]}}
\end{equation}

\begin{equation}
b^{[l]} := b^{[l]} - \alpha \frac{\partial L}{\partial b^{[l]}}
\end{equation}



In [ ]:
def update_parameters(list_w,list_b,list_dw,list_db,alpha):
    
    N_couches = #A COMPLETER
    
    for l in range(N_couches):
        
        list_w[l] = #A COMPLETER
        list_b[l] = #A COMPLETER
    
    return list_w,list_b

**Vérification** : Exécutez la cellule ci-dessous

In [ ]:
#NE PAS MODIFIER

np.random.seed(seed = 2)

X = np.random.rand(5,3)
list_W = [np.random.rand(4,5)-1/2,np.random.rand(4,4)-1/2,np.random.rand(2,4)-1/2]
list_b = [np.random.rand(4,1)-1/2,np.random.rand(4,1)-1/2,np.random.rand(2,1)-1/2]
list_activations = ["ReLU","ReLU","sigmoide"]

Y_pred,caches = feed_forward(X,list_W,list_b,list_activations)

Y = np.array([[0,1,1],[1,0,1]])

list_dw, list_db = backward_propagation(Y_pred,Y,caches,list_activations)

print(update_parameters(list_W,list_b,list_dw,list_db,0.01))

## Initialisation des paramètres

Il nous manque une étape essentielle : l'initialisation des paramètres. Nous allons faire une initialisation aléatoire des paramètres, suivant une loi normale centrée réduite.

**Exercice** : On se donne une structure de réseau de neurones : une liste contenant le nombre de neurones de chaque couche. Le premier élément de la liste est la taille de la donnée d'entrée, les éléments suivant sont le nombre de neurones de chaque couche.

Votre exercice est de fournir une liste initiale de matrices aléatoires de poids W et b, tirés suivant une loi normale centrée réduite, avec les bonnes dimensions. La liste list_neurons contient le nombre de neurones de chaque couche et le premier élément est la dimension des données d'entrée.

Nous rappelons que les matrices des poids sont de forme (nombre de neurones couche l + 1, nombre de neurones couche l). Les vecteurs de biais sont de formes (nombre de neurones couche l). Notons qu'il n'est pas nécessaire d'initialiser aléatoirement les vecteurs de biais. Nous allons ici ne mettre que des zéros.

**Hint** :
- Les fonctions np.random.randn et np.zeros seront utiles


In [ ]:
def initialisation_parameters(list_neurons):
    
    N_couches = len(list_neurons)
    
    list_W = []
    list_b = []
    
    for l in range(N_couches - 1):
        
        list_W.append(#A COMPLETER)
        list_b.append(#A COMPLETER)
    
    return list_W, list_b

**Vérification** : Exécutez la cellule suivante

In [ ]:
#NE PAS MODIFIER

list_neurons = [2,3,3,1]

np.random.seed(seed = 2)

print(initialisation_parameters(list_neurons))

## Et c'est parti pour l'entraînement

Nous allons nous intéresser au problème du XOR (ou exclusif). Prenons des vecteurs à deux dimensions. Si les deux coordonnées ont le même signe, on dira que le vecteur appartient à la classe 1, et 0 si les deux coordonnées sont de signes opposés. En exercice, vous pouvez faire un dessin représentant la situation. On comprend ici qu'un séparateur linéaire ne permettra pas de séparer les deux classes. Nous allons faire un réseau de neurones pour cela. Ci-dessous, je vous construis une base de données.

In [ ]:
#NE PAS MODIFIER

N_train = 200

X_train = 2*np.random.rand(2,N_train) - 1

Y_train = (np.sign(X_train[0,:]) == np.sign(X_train[1,:]))*1

Y_train = np.reshape(Y_train,(1,N_train))

N_test = 100

X_test = 2*np.random.rand(2,N_test) - 1

Y_test = (np.sign(X_test[0,:]) == np.sign(X_test[1,:]))*1

Y_test = np.reshape(Y_test,(1,N_test))

**Exercice** : Initialiser les paramètres d'un réseau d'architecture suivante : [2,5,5,1]. L'entrée est en effet de dimension 2 et la sortie de dimension 1. Donnez aussi les fonctions d'activation pour l'ensemble des couches intermédiaires et sigmoïde pour la dernière sortie (attention, il n'y a que 3 couches).

In [ ]:
list_neurons = #A COMPLETER

list_activations = #A COMPLETER

list_W,list_b = #A COMPLETER

**Exercice** : Effectuez un millier d'époques d'apprentissage avec un taux d'apprentissage de 0.1. A chaque itération, affichez la valeur de la loss function sur la base de test et la base d'apprentissage. Utilisez les fonctions que vous avez définies ci-dessus.

**Hint** :
- A chaque époque, effectuez une propagation forward pass
- Calculez la loss function que vous obtenez et affichez-la (base d'entraînement et base de test)
- Effectuez la back-propagation pour calculer les listes de gradients
- Mettez à jour les paramètres
- Et on boucle !

In [ ]:
N_epoques = 1000

alpha = 0.1

for i in range(N_epoques):
    
    Y_pred_train, caches = #A COMPLETER
    
    Y_pred_test, _ = #A COMPLETER
    
    loss_train = #A COMPLETER
    
    loss_test = #A COMPLETER
    
    print("Epoque" + str(i))
    print("Training loss: " + str(loss_train))
    print("Test loss: " + str(loss_test))
    
    list_dW, list_db = #A COMPLETER
    
    list_W,list_b = #A COMPLETER

Les loss functions devraient décroître si votre apprentissage se passent bien. Ci dessous, vous trouverez la précision sur le jeu de test, ainsi que sur le jeu d'entraînement : on se fixe un seuil de 0.5, si la prédiction est supérieure à 0.5, on l'associe à la classe 1, et à la classe 0 sinon. On compte le nombre de fois où la classe prédite est égale à la classe attendue et on regarde par rapport au nombre d'exemples.

In [ ]:
print("Accuracy sur le jeu de test : " + str(np.sum((Y_pred_test > 0.5) == Y_test)/N_test))

In [ ]:
print("Accuracy sur le jeu d'entraînement : " + str(np.sum((Y_pred_train > 0.5) == Y_train)/N_train))

Ci-dessous pour visualiser vos prédictions sur l'ensemble du carré unité en deux dimensions !

In [ ]:
grid = np.meshgrid(np.linspace(-1.,1.,50),np.linspace(-1,1,50))

X_test_new = np.array([grid[0].flatten(),grid[1].flatten()])

Y_pred_new, _ = feed_forward(X_test_new,list_W,list_b,list_activations)

maps = plt.imshow((Y_pred_new.reshape(grid[0].shape[0],grid[0].shape[1])),extent = (-1,1,-1,1),cmap = "hot",origin = "lower")
plt.scatter(X_test[0,:],X_test[1,:],color = "blue",marker = "x")
plt.colorbar(maps, label = "Sortie du réseau de neurones")